In [1]:
%matplotlib inline

import numpy as np
import scipy.linalg
import matplotlib.pyplot as plt
from numba import jit

In [2]:
@jit(nopython=True)
def get_combination(NOS,NOD):
    combination = np.zeros((NOS,NOD),dtype=np.int64)
    for i in range(NOS):
        for j in range(NOD):
            combination[i,j] = f_combination(i+1,j+1)
    return combination

In [3]:
@jit(nopython=True)
def f_combination(n,k):
    if n<k:
        return 0
    nCk = 1
    for i in range(1,k+1):
        nCk = nCk * (n-k+i)
        nCk = nCk//i
    return nCk

In [4]:
for n in range(5):
    for k in range(n+1):
        print(n,k,f_combination(n,k))
    print()

combination = get_combination(10,5)
print(combination)

0 0 1

1 0 1
1 1 1

2 0 1
2 1 2
2 2 1

3 0 1
3 1 3
3 2 3
3 3 1

4 0 1
4 1 4
4 2 6
4 3 4
4 4 1

[[  1   0   0   0   0]
 [  2   1   0   0   0]
 [  3   3   1   0   0]
 [  4   6   4   1   0]
 [  5  10  10   5   1]
 [  6  15  20  15   6]
 [  7  21  35  35  21]
 [  8  28  56  70  56]
 [  9  36  84 126 126]
 [ 10  45 120 210 252]]


In [5]:
@jit(nopython=True)
def insertion_sort(a,NOD):
    for i in range(2,NOD+1):
        j = i - 1
        temp = a[i-1]
        while a[j-1] > temp:
            a[j] = a[j-1]
            j = j - 1
            if j==0:
                break
        a[j] = temp
    return 0

In [6]:
a = np.arange(16,0,-1)
print(a)
insertion_sort(a,16)
print(a)

[16 15 14 13 12 11 10  9  8  7  6  5  4  3  2  1]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]


In [7]:
@jit(nopython=True)
def inv_list(ni,NOD,combination):
    val_inv_list = ni[0]
    for i in range(2,NOD+1):
        val_inv_list = val_inv_list + combination[ni[i-1]-2,i-1]
    return val_inv_list

In [8]:
combination = get_combination(10,5)
ni = np.arange(10)
print(combination)
print(ni)
for i in range(5):
    val_inv_list = inv_list(ni,i,combination)
    print(i,val_inv_list)

[[  1   0   0   0   0]
 [  2   1   0   0   0]
 [  3   3   1   0   0]
 [  4   6   4   1   0]
 [  5  10  10   5   1]
 [  6  15  20  15   6]
 [  7  21  35  35  21]
 [  8  28  56  70  56]
 [  9  36  84 126 126]
 [ 10  45 120 210 252]]
[0 1 2 3 4 5 6 7 8 9]
0 0
1 0
2 45
3 45
4 45


In [9]:
@jit(nopython=True)
def qsort_w_order(a,o,first,last):
    x = a[(first+last)//2-1]
    i = first
    j = last
    while True:
        while a[i-1] < x:
            i = i + 1
        while x < a[j-1]:
            j = j - 1
        if i >= j:
            break
        t8 = a[i-1];  a[i-1] = a[j-1];  a[j-1] = t8
        t  = o[i-1];  o[i-1] = o[j-1];  o[j-1] = t
        i = i + 1
        j = j - 1
    if first < i - 1:
        qsort_w_order(a,o,first,i-1)
    if j + 1 < last:
        qsort_w_order(a,o,j+1,last)
    return 0

In [10]:
a = np.arange(16,0,-1)
o = np.arange(32,16,-1)
print(a)
print(o)
qsort_w_order(a,o,4,12)
print(a)
print(o)

[16 15 14 13 12 11 10  9  8  7  6  5  4  3  2  1]
[32 31 30 29 28 27 26 25 24 23 22 21 20 19 18 17]
[16 15 14  5  6  7  8  9 10 11 12 13  4  3  2  1]
[32 31 30 21 22 23 24 25 26 27 28 29 20 19 18 17]


In [11]:
## output "ni" is returned
@jit(nopython=True)
def list_fly(t,NOD,NOS,combination):
    ni = np.zeros(NOD,dtype=np.int64)
    s = t
    j = NOS - 1
    for i in range(NOD,1,-1):
        b, j0 = binary_search(s,combination[:,i-1],i,j)
        j = j0 - 1
        ni[i-1] = j0
        s = s - combination[j-1,i-1]
    ni[0] = s
    return ni

In [12]:
## output "ni" is in arguments
@jit(nopython=True)
def list_fly_2(t,NOD,NOS,combination,ni):
    ni[:] = 0
    s = t
    j = NOS - 1
    for i in range(NOD,1,-1):
        b, j0 = binary_search(s,combination[:,i-1],i,j)
        j = j0 - 1
        ni[i-1] = j0
        s = s - combination[j-1,i-1]
    ni[0] = s
    return 0

In [13]:
@jit(nopython=True)
def binary_search(s,list_s,ls,le):
    bmin = ls; bmax = le
    while True:
        b = bmin + (bmax-bmin)//2
        if s < list_s[b-1]:
            bmax = b - 1
        elif list_s[b-1] < s:
            bmin = b + 1
        else:
            bmin = b
            return b, bmin
        if bmin > bmax:
            b = -1
            return b, bmin
    return b, bmin

In [14]:
@jit(nopython=True)
def list_to_state(st_list,NOS):
    string = ""
    for i in range(1,NOS+1):
        if i in st_list:
            string = string + "1" # down
        else:
            string = string + "0" # up
    return string

In [15]:
NOS = 6 # number of sites
NOD = 3 # number of down spins
combination = get_combination(NOS,NOD)
THS = combination[NOS-1,NOD-1] # total Hilbert space
print(combination)
print(THS)
print()

lv = THS
for i in range(1,lv+1):
    st_list = list_fly(i,NOD,NOS,combination)
    st_01 = list_to_state(st_list,NOS)
    st_idx = inv_list(st_list,NOD,combination)
    print(i,st_list,st_01,st_idx)
print()

lv = THS
st_list = np.zeros(NOD,dtype=np.int64)
for i in range(1,lv+1):
    list_fly_2(i,NOD,NOS,combination,st_list)
    st_01 = list_to_state(st_list,NOS)
    st_idx = inv_list(st_list,NOD,combination)
    print(i,st_list,st_01,st_idx)
print()

[[ 1  0  0]
 [ 2  1  0]
 [ 3  3  1]
 [ 4  6  4]
 [ 5 10 10]
 [ 6 15 20]]
20

1 [1 2 3] 111000 1
2 [1 2 4] 110100 2
3 [1 3 4] 101100 3
4 [2 3 4] 011100 4
5 [1 2 5] 110010 5
6 [1 3 5] 101010 6
7 [2 3 5] 011010 7
8 [1 4 5] 100110 8
9 [2 4 5] 010110 9
10 [3 4 5] 001110 10
11 [1 2 6] 110001 11
12 [1 3 6] 101001 12
13 [2 3 6] 011001 13
14 [1 4 6] 100101 14
15 [2 4 6] 010101 15
16 [3 4 6] 001101 16
17 [1 5 6] 100011 17
18 [2 5 6] 010011 18
19 [3 5 6] 001011 19
20 [4 5 6] 000111 20

1 [1 2 3] 111000 1
2 [1 2 4] 110100 2
3 [1 3 4] 101100 3
4 [2 3 4] 011100 4
5 [1 2 5] 110010 5
6 [1 3 5] 101010 6
7 [2 3 5] 011010 7
8 [1 4 5] 100110 8
9 [2 4 5] 010110 9
10 [3 4 5] 001110 10
11 [1 2 6] 110001 11
12 [1 3 6] 101001 12
13 [2 3 6] 011001 13
14 [1 4 6] 100101 14
15 [2 4 6] 010101 15
16 [3 4 6] 001101 16
17 [1 5 6] 100011 17
18 [2 5 6] 010011 18
19 [3 5 6] 001011 19
20 [4 5 6] 000111 20



In [16]:
## bug, not used
## output "nd" is returned
#@jit(nopython=True)
#def j_flip_ni(i,j,n,NOD):
#    nd = np.ones(NOD,dtype=np.int64)
#    for kr in range(NOD,0,-1):
#        if j < n[kr-1]:
#            continue
#        elif j > n[kr-1]:
#            break
#        else:
#            nd[:] = 0
#            break
#    if nd[NOD-1] == 1: # S+_i S-_j
#        for kl in range(1,kr+1):
#            if i == n[kl-1]:
#                break
#        nd[kl-1:kr-1] = n[kl:kr]
#        nd[kr-1] = j
#    else: # S-_i S+_j
#        for kl in range(1,kr+1):
#            if i < n[kl-1]:
#                break
#        nd[kl-1] = i
#        nd[kl:kr] = n[kl-1:kr-1]
#    nd[0:kl-1] = n[0:kl-1]
#    nd[kr:NOD] = n[kr:NOD]
#    return nd

In [17]:
## output "nd" is returned
@jit(nopython=True)
def j_flip_ni(i,j,n,NOD):
    nd = np.ones(NOD,dtype=np.int64)
    kr = NOD
    for _kr in range(NOD,0,-1):
        if j < n[_kr-1]:
            kr = _kr
            continue
        elif j > n[_kr-1]:
            kr = _kr
            break
        else:
            nd[:] = 0
            kr = _kr
            break
    if nd[NOD-1] == 1: # S+_i S-_j
        kl = 1
        for _kl in range(1,kr+1):
            if i == n[_kl-1]:
                kl = _kl
                break
            kl = _kl+1
        nd[kl-1:kr-1] = n[kl:kr]
        nd[kr-1] = j
    else: # S-_i S+_j
        kl = 1
        for _kl in range(1,kr+1):
            if i < n[_kl-1]:
                kl = _kl
                break
            kl = _kl+1
        nd[kl-1] = i
        nd[kl:kr] = n[kl-1:kr-1]
    nd[0:kl-1] = n[0:kl-1]
    nd[kr:NOD] = n[kr:NOD]
    return nd

In [18]:
## output "nd" is in arguments
@jit(nopython=True)
def j_flip_ni_2(i,j,n,NOD,nd):
    nd[:] = 1
    kr = NOD
    for _kr in range(NOD,0,-1):
        if j < n[_kr-1]:
            kr = _kr
            continue
        elif j > n[_kr-1]:
            kr = _kr
            break
        else:
            nd[:] = 0
            kr = _kr
            break
    if nd[NOD-1] == 1: # S+_i S-_j
        kl = 1
        for _kl in range(1,kr+1):
            if i == n[_kl-1]:
                kl = _kl
                break
            kl = _kl+1
        nd[kl-1:kr-1] = n[kl:kr]
        nd[kr-1] = j
    else: # S-_i S+_j
        kl = 1
        for _kl in range(1,kr+1):
            if i < n[_kl-1]:
                kl = _kl
                break
            kl = _kl+1
        nd[kl-1] = i
        nd[kl:kr] = n[kl-1:kr-1]
    nd[0:kl-1] = n[0:kl-1]
    nd[kr:NOD] = n[kr:NOD]
    return 0

In [19]:
NOS = 4 # number of sites
NOD = 2 # number of down spins
combination = get_combination(NOS,NOD)
THS = combination[NOS-1,NOD-1] # total Hilbert space
print(combination)
print(THS)
print()

for sp in range(1,THS+1):
    st_list = list_fly(sp,NOD,NOS,combination)
    print(sp,st_list,list_to_state(st_list,NOS))
#    st_idx = inv_list(st_list,NOD,combination)
#    print(sp,st_list,list_to_state(st_list,NOS),st_idx)
    for i in range(1,NOS+1):
        for j in range(i+1,NOS+1):
            nd = j_flip_ni(i,j,st_list,NOD)
            print(i,j,nd,list_to_state(nd,NOS))
#            nd_idx = inv_list(nd,NOD,combination)
#            print(i,j,nd,list_to_state(nd,NOS),nd_idx)
    print("--")
print()

st_list = np.zeros(NOD,dtype=np.int64)
nd = np.zeros(NOD,dtype=np.int64)
for sp in range(1,THS+1):
    list_fly_2(sp,NOD,NOS,combination,st_list)
    print(sp,st_list,list_to_state(st_list,NOS))
#    st_idx = inv_list(st_list,NOD,combination)
#    print(sp,st_list,list_to_state(st_list,NOS),st_idx)
    for i in range(1,NOS+1):
        for j in range(i+1,NOS+1):
            j_flip_ni_2(i,j,st_list,NOD,nd)
            print(i,j,nd,list_to_state(nd,NOS))
#            nd_idx = inv_list(nd,NOD,combination)
#            print(i,j,nd,list_to_state(nd,NOS),nd_idx)
    print("--")
print()

[[1 0]
 [2 1]
 [3 3]
 [4 6]]
6

1 [1 2] 1100
1 2 [1 1] 1000
1 3 [2 3] 0110
1 4 [2 4] 0101
2 3 [1 3] 1010
2 4 [1 4] 1001
3 4 [1 2] 1100
--
2 [1 3] 1010
1 2 [2 3] 0110
1 3 [1 1] 1000
1 4 [3 4] 0011
2 3 [1 2] 1100
2 4 [1 3] 1010
3 4 [1 4] 1001
--
3 [2 3] 0110
1 2 [1 3] 1010
1 3 [1 2] 1100
1 4 [2 3] 0110
2 3 [2 2] 0100
2 4 [3 4] 0011
3 4 [2 4] 0101
--
4 [1 4] 1001
1 2 [2 4] 0101
1 3 [3 4] 0011
1 4 [1 1] 1000
2 3 [1 4] 1001
2 4 [1 2] 1100
3 4 [1 3] 1010
--
5 [2 4] 0101
1 2 [1 4] 1001
1 3 [2 4] 0101
1 4 [1 2] 1100
2 3 [3 4] 0011
2 4 [2 2] 0100
3 4 [2 3] 0110
--
6 [3 4] 0011
1 2 [3 4] 0011
1 3 [1 4] 1001
1 4 [1 3] 1010
2 3 [2 4] 0101
2 4 [2 3] 0110
3 4 [3 3] 0010
--

1 [1 2] 1100
1 2 [1 1] 1000
1 3 [2 3] 0110
1 4 [2 4] 0101
2 3 [1 3] 1010
2 4 [1 4] 1001
3 4 [1 2] 1100
--
2 [1 3] 1010
1 2 [2 3] 0110
1 3 [1 1] 1000
1 4 [3 4] 0011
2 3 [1 2] 1100
2 4 [1 3] 1010
3 4 [1 4] 1001
--
3 [2 3] 0110
1 2 [1 3] 1010
1 3 [1 2] 1100
1 4 [2 3] 0110
2 3 [2 2] 0100
2 4 [3 4] 0011
3 4 [2 4] 0101
--
4 [1 4] 1001


In [20]:
for f1 in [True,False]:
    for f2 in [True,False]:
        f3 = f1^f2
        print(f1,f2,f3)

True True False
True False True
False True True
False False False


In [21]:
## output "Ham" is returned
@jit(nopython=True)
def make_full_hamiltonian(lv,combination,NOD,NOxxz,p_xxz,sJint,NOS):
    Ham = np.zeros((lv,lv),dtype=np.float64)
    for i in range(1,lv+1):
        st_list = list_fly(i,NOD,NOS,combination)
        for j in range(1,NOxxz+1):
            f1 = p_xxz[0,j-1] in st_list
            f2 = p_xxz[1,j-1] in st_list
            if f1^f2:
                Ham[i-1,i-1] = Ham[i-1,i-1] - sJint[j-1,1]
                ni = j_flip_ni(p_xxz[0,j-1],p_xxz[1,j-1],st_list,NOD)
                id = inv_list(ni,NOD,combination)
                Ham[i-1,id-1] = Ham[i-1,id-1] + sJint[j-1,0]
            else:
                Ham[i-1,i-1] = Ham[i-1,i-1] + sJint[j-1,1]
    return Ham

In [22]:
## output "Ham" is in arguments
@jit(nopython=True)
def make_full_hamiltonian_2(lv,Ham,combination,NOD,NOxxz,p_xxz,sJint,NOS):
    st_list = np.zeros(NOD,dtype=np.int64)
    ni = np.zeros(NOD,dtype=np.int64)
    for i in range(1,lv+1):
        list_fly_2(i,NOD,NOS,combination,st_list)
        for j in range(1,NOxxz+1):
            f1 = p_xxz[0,j-1] in st_list
            f2 = p_xxz[1,j-1] in st_list
            if f1^f2:
                Ham[i-1,i-1] = Ham[i-1,i-1] - sJint[j-1,1]
                j_flip_ni_2(p_xxz[0,j-1],p_xxz[1,j-1],st_list,NOD,ni)
                id = inv_list(ni,NOD,combination)
                Ham[i-1,id-1] = Ham[i-1,id-1] + sJint[j-1,0]
            else:
                Ham[i-1,i-1] = Ham[i-1,i-1] + sJint[j-1,1]
    return 0

In [23]:
NOS = 4 # number of sites
NOD = 2 # number of down spins
NOxxz = NOS # number of XXZ interaction
combination = get_combination(NOS,NOD)
THS = combination[NOS-1,NOD-1] # total Hilbert space
print(combination)
print(THS)

p_xxz = np.zeros((2,NOxxz),dtype=np.int64)
Jint = np.zeros((NOxxz,2),dtype=np.float64) # Jint[NOxxz,0] --> Jint_x, Jint[NOxxz,1] --> Jint_z
sJint = np.zeros((NOxxz,2),dtype=np.float64) # sJint[NOxxz,0] --> sJint_x, sJint[NOxxz,1] --> sJint_z
for i in range(NOS):
    p_xxz[0,i] = i%NOS+1
    p_xxz[1,i] = (i+1)%NOS+1
    if p_xxz[0,i] > p_xxz[1,i]: # assume i<j for pair (i,j)
        tmp = p_xxz[0,i]
        p_xxz[0,i] = p_xxz[1,i]
        p_xxz[1,i] = tmp
Jint[:,:] = 1.0
sJint[:,0] = 0.5 * Jint[:,0]
sJint[:,1] = 0.25 * Jint[:,1]
print(p_xxz)
print(Jint)
print(sJint)
print()

Ham = make_full_hamiltonian(THS,combination,NOD,NOxxz,p_xxz,sJint,NOS)
print(Ham)
for i in range(THS):
    for j in range(THS):
        if np.abs(Ham[i,j]) > 1e-6:
            print(i,j,Ham[i,j])
print()

Ham = np.zeros((THS,THS),dtype=np.float64)
make_full_hamiltonian_2(THS,Ham,combination,NOD,NOxxz,p_xxz,sJint,NOS)
print(Ham)
for i in range(THS):
    for j in range(THS):
        if np.abs(Ham[i,j]) > 1e-6:
            print(i,j,Ham[i,j])
print()

[[1 0]
 [2 1]
 [3 3]
 [4 6]]
6
[[1 2 3 1]
 [2 3 4 4]]
[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]
[[0.5  0.25]
 [0.5  0.25]
 [0.5  0.25]
 [0.5  0.25]]

[[ 0.   0.5  0.   0.   0.5  0. ]
 [ 0.5 -1.   0.5  0.5  0.   0.5]
 [ 0.   0.5  0.   0.   0.5  0. ]
 [ 0.   0.5  0.   0.   0.5  0. ]
 [ 0.5  0.   0.5  0.5 -1.   0.5]
 [ 0.   0.5  0.   0.   0.5  0. ]]
0 1 0.5
0 4 0.5
1 0 0.5
1 1 -1.0
1 2 0.5
1 3 0.5
1 5 0.5
2 1 0.5
2 4 0.5
3 1 0.5
3 4 0.5
4 0 0.5
4 2 0.5
4 3 0.5
4 4 -1.0
4 5 0.5
5 1 0.5
5 4 0.5

[[ 0.   0.5  0.   0.   0.5  0. ]
 [ 0.5 -1.   0.5  0.5  0.   0.5]
 [ 0.   0.5  0.   0.   0.5  0. ]
 [ 0.   0.5  0.   0.   0.5  0. ]
 [ 0.5  0.   0.5  0.5 -1.   0.5]
 [ 0.   0.5  0.   0.   0.5  0. ]]
0 1 0.5
0 4 0.5
1 0 0.5
1 1 -1.0
1 2 0.5
1 3 0.5
1 5 0.5
2 1 0.5
2 4 0.5
3 1 0.5
3 4 0.5
4 0 0.5
4 2 0.5
4 3 0.5
4 4 -1.0
4 5 0.5
5 1 0.5
5 4 0.5

